## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

Import my library:

In [ ]:
import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import dataset_comp

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)

# Mass Distribution of Subhalos

To visualize the mass distribution, at $z=0$, I plot a simple subhalo count accumulation curve, which at any point on the mass axis gives the total number of subhalos (or of satellite or isolated galaxies) with masses larger than the given mass. Mass is measured by $v_\mathrm{max} = \max_{r} \sqrt{\frac{G M(<r)}{r}}$.

## Motivation

Plenty of interesting and instructive observations can be made from the mass distribution figure alone. For the curv-p08* simulation, we expect find an indication of inhibited structure formation towards the small scales, relative to the plain-lcdm. However, on the larger scales, the curves from different simulations should approach each other (although random effects from low number counts also come into play). 

But the absence of small-scale power in the initial physical power spectrum means that the component of power that is due to numerical noise also becomes more significant --- and, indeed, will dominate on small enough scales (shot noise becomes more relevant towards the resolution limits of the simulation). This is expected to be visible in the mass function as well.

Furthermore, the mass functions satellite galaxies and isolated galaxies in any simulations should be expected to differ. 

From the single plot alone, it is impossible to make the connection between any particular feature of the mass function and any of the above-mentioned potential causes. Of course, we are also only looking at a single simulated instance of the LG.

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation:

Create a dictionary of the datasets from each simulation. 

In [ ]:
snap_id = 127
sim_ids = ["V1_MR_fix", "V1_MR_curvaton_p082_fix", "V1_LR_fix", "V1_LR_curvaton_p082_fix", "V1_LR_curvaton_p084_fix"]
names = ["plain-LCDM", "spec-p082", "plain-LCDM-LR", "spec-p082-LR", "spec-p084-LR"]
sim_colors = [['black', 'gray', 'lightgray'], ['red', 'pink', 'lightpink'],
              ['gray'], ['pink'], ['lightblue']]

# Define M31 and MW in each simulation:
m31 = [(1,0), (1,0), (1,0), (1,0), (1,0)]
mw = [(2,0), (1,1), (2,0), (1,1), (1,0)]

Choose how to distinguish between satellite and isolated galaxies:

In [ ]:
distinction = 'by_r'
maxdi = 2000 # Maximum distance from LG centre for isolated

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns, col in zip(names, sim_ids, m31, mw, sim_colors):
    data[name] = {"snapshot": snapshot.Snapshot(sim_id, snap_id, name=name),
                  "M31_identifier": m31_ns,
                  "MW_identifier": mw_ns,
                  'PlotStyle': {
                      'Color': col
                  }}

In [ ]:
z = data["plain-LCDM"]["snapshot"].get_attribute("Redshift", "Header")

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for key, sim_data in data.items():    
    # Get data:
    snap = sim_data["snapshot"]
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
        
    # Add M31 and MW to the dictionary:
    idx_m31 = snap.index_of_halo(sim_data["M31_identifier"][0], sim_data["M31_identifier"][1])
    sim_data["M31"] = {"Vmax": vmax[idx_m31]}
    idx_mw = snap.index_of_halo(sim_data["MW_identifier"][0], sim_data["MW_identifier"][1])
    sim_data["MW"] = {"Vmax": vmax[idx_mw]}
        
    # Split into satellites:
    if distinction == "by_r":
        masks_sat, mask_isol = dataset_comp.split_satellites_by_distance(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    elif distinction == "by_GN":
        masks_sat, mask_isol = dataset_comp.split_satellites_by_group_number(
            snap, sim_data["M31_identifier"], sim_data["MW_identifier"])
    
    # Compute masking arrays:
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    mask_nonzero_vmax = dataset_comp.prune_vmax(snap)

    # Sort by vmax and add a dummy point with very small vmax 
    # (to continue the curves to the y-axis):
    sort_idx = np.argsort(vmax)
    vmax = np.concatenate([[0.01], vmax[sort_idx]])
    sim_data["Vmax"] = vmax
    mask_m31 = np.concatenate([[True], masks_sat[0][sort_idx]])
    mask_mw = np.concatenate([[True], masks_sat[1][sort_idx]])
    mask_sat = np.logical_or(mask_m31, mask_mw)
    mask_isol = np.concatenate([[True], mask_isol[sort_idx]])
    mask_lum = np.concatenate([[True], mask_lum[sort_idx]])
    mask_dark = np.concatenate([[True], mask_dark[sort_idx]])
    mask_nonzero_vmax = np.concatenate([[True], mask_nonzero_vmax[sort_idx]])

    # Add selections (masking arrays):
    data[key]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Satellite': mask_sat,
        'Isolated': mask_isol,
        'Luminous': mask_lum,
        'Dark': mask_dark
    }

## Plot Only Total Counts

In [ ]:
# Set some parameters:
x_down = 7; x_up = 110
y_down = 1; y_up = 1000

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8,4), sharey="row")
plt.subplots_adjust(wspace=0)

# Set axis:
axes[0].set_ylim(y_down, y_up)
axes[0].set_ylabel('$N(>v_{\mathrm{max}})$')
for ax in axes:
    ax.set_xlim(x_down, x_up)
    ax.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')
    ax.set_xscale('log')
    ax.set_yscale('log')

# Add redshift:
# z_x = 10 ** (np.log10(x_up) - 9/10 * (np.log10(x_up)-np.log10(x_down)))
# z_y = 10 ** (np.log10(y_up) - 1/10 * (np.log10(y_up)-np.log10(y_down)))
# axes[0].text(z_x, z_y, "z = {:.2f}".format(z))
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')


# First, plot LR on the background:
for (name, entry) in data.items():
    
    if name.split("-")[-1] != "LR":
        continue
               
    vmax = entry["Vmax"]
        
    # ISOLATED GALAXIES
    # -----------------
    
    mask_isol = entry['Selections']['Isolated']
    cnt_all = np.arange(1, np.sum(mask_isol) + 1)[::-1] 
    axes[1].plot(vmax[mask_isol], cnt_all,
                 c=entry['PlotStyle']['Color'][0], linestyle='solid')


    # SATELLITES
    # ----------
        
    if name.split("-")[1] == "p084":
        continue

    mask_sat = entry['Selections']['Satellite']
    cnt = np.arange(1, np.sum(mask_sat) + 1)[::-1] 
    axes[0].plot(vmax[mask_sat], cnt,
                 c=entry['PlotStyle']['Color'][0], linestyle='solid')

    
# Then, plot MR:
for (name, entry) in data.items():
    
    if name.split("-")[-1] == "LR":
        continue
        
    vmax = entry["Vmax"]
    
    # SATELLITES
    # ----------
    
    mask_sat = entry['Selections']['Satellite']
    cnt_all = np.arange(1, np.sum(mask_sat) + 1)[::-1]
    axes[0].plot(vmax[mask_sat], cnt_all,
                 c=entry['PlotStyle']['Color'][0], linestyle='solid')
    
    
    # ISOLATED GALAXIES
    # -----------------
    
    mask_all = entry['Selections']['Isolated']
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
    axes[1].plot(vmax[mask_all], cnt_all,
                 c=entry['PlotStyle']['Color'][0], linestyle='solid')
        
        
# Make dummy plots for the legend:   
l = []
for entry in data.values():
    l_dum, = axes[1].plot([], [], c=entry['PlotStyle']['Color'][0], linestyle="solid")
    l.append(l_dum)
sim_legend = axes[1].legend(l, list(data.keys()), loc="upper right")
axes[1].add_artist(sim_legend)
    
plt.tight_layout()

In [ ]:
axes[0].axvline(10, c='purple', linestyle='dotted')
axes[1].axvline(15, c='purple', linestyle='dotted')
fig

### Save the Figure

In [ ]:
filename = "mass_distribution_total.png"
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')

## Plot Luminous and Dark

In [ ]:
# Set some parameters:
x_down_sat = 10; x_up_sat = 110
x_down_isol = 10; x_up_isol = 110
y_down = 1; y_up = 500

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8,4), sharey="row")
plt.subplots_adjust(wspace=0)

# Set axis:
axes[0].set_xlim(x_down_sat, x_up_sat)
axes[1].set_xlim(x_down_isol, x_up_isol)
axes[0].set_ylim(y_down, y_up)
axes[0].set_ylabel('$N(>v_{\mathrm{max}})$')
for ax in axes:
    ax.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')
    ax.set_xscale('log')
    ax.set_yscale('log')

# Add redshift:
# z_x = 10 ** (np.log10(x_up) - 9/10 * (np.log10(x_up)-np.log10(x_down)))
# z_y = 10 ** (np.log10(y_up) - 1/10 * (np.log10(y_up)-np.log10(y_down)))
# axes[0].text(z_x, z_y, "z = {:.2f}".format(z))
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')
    
# Then, plot MR:
data_mr = {key: data[key] for key in data.keys() 
           & {'plain-LCDM', 'spec-p082'}}
for name, entry in data_mr.items():
        
    vmax = entry["Vmax"]
    
    # SATELLITES
    # ----------

    mask_sat = entry['Selections']['Satellite']
    mask_lum = np.logical_and(mask_sat,
                              entry["Selections"]["Luminous"])
    
    # Plot all luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0].plot(vmax[mask_lum], cnt_lum,
                 c=entry['PlotStyle']['Color'][0], linestyle='dashed')
    
    # Plot all satellites, luminous and dark:
    cnt_all = np.arange(1, np.sum(mask_sat) + 1)[::-1]
    axes[0].plot(vmax[mask_sat], cnt_all,
                 c=entry['PlotStyle']['Color'][0], linestyle='solid')
  
    # Fill area of dark satellites:
    axes[0].fill(
        np.append(vmax[mask_lum], vmax[mask_sat][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=entry['PlotStyle']['Color'][2],
        alpha=0.3
    )
        
    # Plot M31 luminous satellites:
#     mask_m31_lum = entry["Selections"]["M31"]["Luminous"]
#     cnt_m31_lum = np.arange(1, np.sum(mask_m31_lum) + 1)[::-1] 
#     axes[0].plot(vmax[mask_m31_lum], cnt_m31_lum, 
#                  c=color[i][0], linestyle='dotted', label='{} luminous'.format(name))
    

    # ISOLATED GALAXIES
    # -----------------
    
    mask_all = entry['Selections']['Isolated']
    mask_lum = np.logical_and(mask_all,
                              entry["Selections"]["Luminous"])
    
    # Plot luminous isolated:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1].plot(vmax[mask_lum], cnt_lum,
                 c=entry['PlotStyle']['Color'][0], linestyle='dashed')
    
    
    # Plot all isolated:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
    axes[1].plot(vmax[mask_all], cnt_all,
                 c=entry['PlotStyle']['Color'][0], linestyle='solid')
  
    # Fill area of dark isolated:
    axes[1].fill(
        np.append(vmax[mask_lum], vmax[mask_all][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=entry['PlotStyle']['Color'][2],
        alpha=0.3
    )

    
        
# Make dummy plots for the legend:   
l = []
for entry in data_mr.values():
    l_dum, = axes[1].plot([], [], c=entry['PlotStyle']['Color'][0], linestyle="solid")
    l.append(l_dum)
sim_legend = axes[1].legend(l, list(data_mr.keys()), loc="upper right")
axes[1].add_artist(sim_legend)

axes[0].plot([], [], c='k', linestyle="solid", label="All")
axes[0].plot([], [], c='k', linestyle="dashed", label="Luminous")
axes[0].legend(loc="upper right")
    
plt.tight_layout()

### Save the Figure

In [ ]:
filename = "mass_distribution_MR_only.png"
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')

## Plot All

In [ ]:
# Set some parameters:
x_down_sat = 10; x_up_sat = 110
x_down_isol = 10; x_up_isol = 110
y_down = 1; y_up = 500

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8,4), sharey="row")
plt.subplots_adjust(wspace=0)

# Set axis:
axes[0].set_ylim(y_down, y_up)
axes[0].set_ylabel('$N(>v_{\mathrm{max}})$')
for ax in axes:
    ax.set_xlim(x_down, x_up)
    ax.set_xlabel('$v_{\mathrm{max}}[\mathrm{km s^{-1}}]$')
    ax.set_xscale('log')
    ax.set_yscale('log')

# Add redshift:
# z_x = 10 ** (np.log10(x_up) - 9/10 * (np.log10(x_up)-np.log10(x_down)))
# z_y = 10 ** (np.log10(y_up) - 1/10 * (np.log10(y_up)-np.log10(y_down)))
# axes[0].text(z_x, z_y, "z = {:.2f}".format(z))
    
axes[0].set_title('Satellite galaxies')
axes[1].set_title('Isolated galaxies')

# Add scatter plots:
color = [['black', 'gray', 'lightgray'], ['red', 'pink', 'lightpink'],
         ['gray'], ['pink'],
         ['lightblue']]

# First, plot LR on the background:
for i, (name, entry) in enumerate(data.items()):
    
    if name.split("-")[-1] != "LR":
        continue
               
    vmax = entry["Vmax"]
        
    # ISOLATED GALAXIES
    # -----------------
    
    mask_isol = entry['Selections']['Isolated']
    mask_lum = np.logical_and(mask_isol,
                              entry["Selections"]["Luminous"])
    print(np.sum(mask_isol))
    
    # Plot luminous isolated:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all isolated:
    cnt_all = np.arange(1, np.sum(mask_isol) + 1)[::-1] 
    axes[1].plot(vmax[mask_isol], cnt_all,
                 c=color[i][0], linestyle='solid')


    # SATELLITES
    # ----------
        
    if name.split("-")[1] == "p084":
        continue

    mask_sat = entry['Selections']['Satellite']
    mask_lum = np.logical_and(mask_sat,
                              entry["Selections"]["Luminous"])
    
    # Plot luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all satellites:
    cnt = np.arange(1, np.sum(mask_sat) + 1)[::-1] 
    axes[0].plot(vmax[mask_sat], cnt,
                 c=color[i][0], linestyle='solid')

    
# Then, plot MR:
for i, (name, entry) in enumerate(data.items()):
    
    if name.split("-")[-1] == "LR":
        continue
        
    vmax = entry["Vmax"]
    
    # SATELLITES
    # ----------
    

    mask_sat = entry['Selections']['Satellite']
    mask_lum = np.logical_and(mask_sat,
                              entry["Selections"]["Luminous"])
    
    # Plot all luminous satellites:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[0].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    # Plot all satellites, luminous and dark:
    cnt_all = np.arange(1, np.sum(mask_sat) + 1)[::-1]
    axes[0].plot(vmax[mask_sat], cnt_all,
                 c=color[i][0], linestyle='solid')
  
    # Fill area of dark satellites:
    axes[0].fill(
        np.append(vmax[mask_lum], vmax[mask_sat][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=color[i][2],
        alpha=0.3
    )
        
    # Plot M31 luminous satellites:
#     mask_m31_lum = entry["Selections"]["M31"]["Luminous"]
#     cnt_m31_lum = np.arange(1, np.sum(mask_m31_lum) + 1)[::-1] 
#     axes[0].plot(vmax[mask_m31_lum], cnt_m31_lum, 
#                  c=color[i][0], linestyle='dotted', label='{} luminous'.format(name))
    

    # ISOLATED GALAXIES
    # -----------------
    
    mask_all = entry['Selections']['Isolated']
    mask_lum = np.logical_and(mask_all,
                              entry["Selections"]["Luminous"])
    
    # Plot luminous isolated:
    cnt_lum = np.arange(1, np.sum(mask_lum) + 1)[::-1] 
    axes[1].plot(vmax[mask_lum], cnt_lum,
                 c=color[i][0], linestyle='dashed')
    
    
    # Plot all isolated:
    cnt_all = np.arange(1, np.sum(mask_all) + 1)[::-1]
    axes[1].plot(vmax[mask_all], cnt_all,
                 c=color[i][0], linestyle='solid')
  
    # Fill area of dark isolated:
    axes[1].fill(
        np.append(vmax[mask_lum], vmax[mask_all][::-1]),
        np.append(cnt_lum, cnt_all[::-1]),
        color=color[i][2],
        alpha=0.3
    )

    
axes[1].axvspan(x_down_isol, 15, alpha=0.5, color='gray', hatch='X')
            
# Make dummy plots for the legend:   
l = []
for c in color:
    l_dum, = axes[1].plot([], [], c=c[0], linestyle="solid")
    l.append(l_dum)
sim_legend = axes[1].legend(l, list(data.keys()), loc="upper right")
axes[1].add_artist(sim_legend)

axes[0].plot([], [], c=color[0][0], linestyle="solid", label="All")
axes[0].plot([], [], c=color[0][0], linestyle="dashed", label="Luminous")
axes[0].legend(loc="upper right")
    
plt.tight_layout()

### Save the Figure

In [ ]:
filename = "mass_distribution.png"
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig.savefig(filename, dpi=300, bbox_inches='tight')